In [15]:
#pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# pip install pip install transformers

In [1]:
import torch
from transformers import (
    AdamW,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
    AutoConfig
)

In [2]:
from src.utils import set_seed
from src.trainer import SimcseTrainer
from src.dataset import DATASET_MAPPING_DICT
from src.utils import PreprocessorFactory 
from src.utils import get_model_argparse
from src.model import MODEL_MAPPING_DICT
from src.model import CONFIG_MAPPING_DICT
from src.logger import Experi_Logger
from config.nli_config import nli_parser_model_args

In [3]:
import os
import logging

In [4]:
from skt.vault_utils import get_secrets
proxies = get_secrets('proxies')

In [5]:
import os 
import numpy as np
import torch

In [6]:
os.environ['http_proxy'] = proxies['http']
os.environ['https_proxy'] = proxies['https']

In [7]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# LOAD ARGS FOR EXPERI SETTING

In [8]:
args = nli_parser_model_args()

In [9]:
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.output_dir = f'/home/x1112436/result/faq/modelfile/{args.pretrained_model}'
args.log_dir = f'/home/x1112436/result/faq/log/{args.pretrained_model}'
args.experiments_path = f'/home/x1112436/result/faq/experiment/{args.pretrained_model}/experiment.csv'
args.is_preprocessed = True
args.valid_first = False
args.data_type='triple'
args.loss= 'TripletLoss'
args.margin = 0.7
args.num_train_epochs = 20
args.model_max_len = 50
args.valid_first = True

In [10]:
print(args.log_dir)

/home/x1112436/result/faq/log/klue/roberta-large


In [11]:
args.pretrained_model = '/home/x1112436/model_file/faq_sent_roberta/sent_roberta'

In [12]:
model = MODEL_MAPPING_DICT['sent_roberta'].from_pretrained(
    args.pretrained_model, **vars(args), 
)
tokenizer = AutoTokenizer.from_pretrained(args.pretrained_model)

# LOAD RESOURCE

In [13]:
# added_tokens = []
# with open('./data/add_tokens.csv', 'r') as f:
#     for data in f:
#         added_tokens.append(data.strip())

In [14]:
added_tokens = [
 'NUGU',
 'wavve',
 'All케어플러스3',
 '컬러링',
 'T플랜',
 'baro',
 '5G',
 'SMS',
 'T파이',
 '티파이'
]

In [15]:
# add tokens to tokenizer and resize embeddings

In [16]:
new_tokens = set(added_tokens) - set(tokenizer.vocab.keys())

In [17]:
tokenizer.add_tokens(list(new_tokens), special_tokens=False)

10

In [18]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32010, 1024)

In [19]:
train_dataset = []
with open('./data/train.csv', 'r') as f:
    for data in f:
        line = data.strip().split('\t')
        train_dataset.append(line)

In [20]:
preprocessor = PreprocessorFactory(data_type=args.data_type)

In [21]:
faq_data = preprocessor.preprocess(
                  data_path='./data/train.csv',
                  tokenizer=tokenizer, 
                  save_path=None,
                  header=False
)

In [22]:
train_dataset = DATASET_MAPPING_DICT['Unsup_simcse']
train_dataset = train_dataset(args=args, features=faq_data, max_length=args.model_max_len, tokenizer=tokenizer)

# validation set

In [23]:
faq_val_data = preprocessor.preprocess(
                  data_path='./data/val.csv',
                  tokenizer=tokenizer, 
                  save_path=None,
                  header=False
)

In [24]:
val_dataset = DATASET_MAPPING_DICT['Unsup_simcse']
val_dataset = val_dataset(args=args, features=faq_val_data, max_length=args.model_max_len, tokenizer=tokenizer)

# Trainer

In [25]:
trainer = SimcseTrainer(args=args, logger=logging)

In [26]:
trainer.model_setting(model_type=args.model_type, train_dataset=train_dataset, model=model, tokenizer=tokenizer)

loss: TripletLoss


/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer.train(model=model, tokenizer=tokenizer, train_dataset=train_dataset, val_dataset=val_dataset, model_type=args.model_type)

loss: TripletLoss


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/2200 [00:00<?, ?it/s]

In [ ]:
import time
time.sleep(21600)